In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

In [2]:
with open("./dataset.csv", 'r') as csvfile:
    print(f"First line (header) looks like this:\n\n{csvfile.readline()}")
    print(f"Each data point looks like this:\n\n{csvfile.readline()}")

First line (header) looks like this:

label,text

Each data point looks like this:

1, Saya suka anime



In [3]:
csv_filename = './dataset.csv'
df = pd.read_csv(csv_filename)

In [4]:
# Pisahkan data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

KeyError: 'labelaaa'

In [ ]:
# Tokenisasi dan pengindeksan kata
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
# Konversi teks ke dalam urutan angka
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

In [ ]:
# Padding agar panjang teks seragam
max_length = max(len(seq) for seq in X_train_sequences)
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding='post')

In [ ]:
# Model jaringan saraf tiruan sederhana
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=32, input_length=max_length))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Kompilasi model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Latih model
model.fit(X_train_padded, y_train, epochs=5, validation_data=(X_test_padded, y_test))

Epoch 1/5
1/1 [==============================] - 3s 3s/step - loss: 0.6956 - accuracy: 0.3333 - val_loss: 0.6907 - val_accuracy: 0.5000
Epoch 2/5
1/1 [==============================] - 0s 24ms/step - loss: 0.6913 - accuracy: 0.8333 - val_loss: 0.6995 - val_accuracy: 0.0000e+00
Epoch 3/5
1/1 [==============================] - 0s 22ms/step - loss: 0.6871 - accuracy: 0.8333 - val_loss: 0.7084 - val_accuracy: 0.0000e+00
Epoch 4/5
1/1 [==============================] - 0s 24ms/step - loss: 0.6829 - accuracy: 0.6667 - val_loss: 0.7175 - val_accuracy: 0.0000e+00
Epoch 5/5
1/1 [==============================] - 0s 24ms/step - loss: 0.6787 - accuracy: 0.6667 - val_loss: 0.7267 - val_accuracy: 0.0000e+00


In [ ]:
# Evaluasi performa model
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f'Accuracy: {accuracy:.2f}')

1/1 [==============================] - 0s 18ms/step - loss: 0.7267 - accuracy: 0.0000e+00
Accuracy: 0.00


In [ ]:
def predict_wibu(input_text):
    # Tokenisasi dan konversi teks ke dalam urutan angka
    input_sequence = tokenizer.texts_to_sequences([input_text])
    input_padded = pad_sequences(input_sequence, maxlen=max_length, padding='post')

    # Lakukan prediksi menggunakan model
    prediction = model.predict(input_padded)[0, 0]

    # Ambil keputusan berdasarkan threshold tertentu
    threshold = 0.5
    label = 1 if prediction >= threshold else 0

    return label, prediction

# Contoh pengujian dengan input baru
input_text_new = "Saya tidak suka anime."
label, confidence = predict_wibu(input_text_new)

# Tampilkan hasil prediksi
if label == 1:
    print(f'Input: "{input_text_new}" adalah Wibu dengan tingkat kepercayaan {confidence:.2%}.')
else:
    print(f'Input: "{input_text_new}" bukan Wibu dengan tingkat kepercayaan {confidence:.2%}.')


1/1 [==============================] - 1s 552ms/step
Input: "Saya tidak suka anime." bukan Wibu dengan tingkat kepercayaan 48.42%.
